In [ ]:
import numpy as np 
import pandas as pd 

# Lectura de los csv

In [ ]:
df_pageviews = pd.read_csv("./data/pageviews.csv",
                   parse_dates=["FEC_EVENT"])
df_devicedata = pd.read_csv("./data/device_data.csv",
                   parse_dates=["FEC_EVENT"])
df_conversiones = pd.read_csv("./data/conversiones.csv")

# Creación del universo

In [ ]:
def universo_train(df):
    users = pd.DataFrame()
    users['user_id'] = df.USER_ID.unique()
    users['key'] = 1
    snapshots = pd.DataFrame({'snapshot_mes':[1,2,3,4,5,6,7,8,9,12]})
    snapshots['key'] = 1
    return pd.merge(users, snapshots, how='left', on = 'key').drop(['key'], axis=1)

In [ ]:
universo_train(df_pageviews)

# Features

In [ ]:
def feature_X(df):
    

In [ ]:
def feature_X1(df):

# Armar dataset

In [ ]:
universo = feature_X(universo)
universo = feature_X1(universo)

# Target

In [ ]:
def target(df):
    df = pd.merge(df, df_conversiones, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df.mes.fillna(value= 99, inplace=True)
    df = df.astype({'mes': 'int32'}) 

    df['target'] = 0
    df.loc[df.mes == df.snapshot_mes +1, 'target'] = 1
    df.loc[df.mes == df.snapshot_mes +2, 'target'] = 1
    df.loc[df.mes == df.snapshot_mes +3, 'target'] = 1

    return (df.groupby(['user_id', 'snapshot_mes'])['target'].max()).to_frame().reset_index()


In [ ]:
target(universo)